# Композиция типов встраиванием структур

Как мы уже говорили в главе про структуры, Go поддерживает встраивание одной структуры в другую. Встраивание также **переносит**(promote) методы встраиваемого типа в тип, в который происходит встраивание. Рассмотрим следующий пример:

In [1]:
import (
    "math"
    "image/color"
    "fmt"
)

type Point struct {
    X float64
    Y float64
}

// Метод для типа Point.
func (p Point) Distance(q Point) float64 {
    return math.Hypot(q.X - p.X, q.Y - p.Y)
}

// Метод с получателем указателем.
func (p *Point) ScaleBy(factor float64) {
    p.X *= factor
    p.Y *= factor
}

// Встраиваем тип Point в тип ColoredPoint.
type ColoredPoint struct {
    Point
    Color color.RGBA
}

var p = ColoredPoint{Point{0,0}, color.RGBA{0xFF, 0, 0, 0xFF}}
var q = ColoredPoint{Point{3,4}, color.RGBA{0, 0xFF, 0, 0xFF}}
fmt.Println(p.Distance(q.Point))
q.ScaleBy(2)
fmt.Sprint(q.Point)

5
{6 8}


Как видно из этого примера для переменной типа ColoredPoint мы смогли вызвать метод Distance, хотя у этого типа нет такого метода. Это стало возможным благодаря встраиванию типа Point в тип ColoredPoint: для типа Point объявлен такой метод.

Такой способ формирования типа называется **композицией**. Композиция позволяет создать тип, объединив в нём с помощью встраивания несколько полей различных типов, каждый из которых предоставляет свои методы.

Тут также важно понять, что ColoredPoint **не является**("is a") Point, а тем более не является наследником типа Point. Т.е. ColoredPoint нельзя использовать в тех местах, где можно использовать Point. Поэтому при вызове метода Distance мы указали в качестве агрумента q.Point, т.к. просто q выдало бы ошибку компиляции несоответствия типов. Правильней будет сказать, что ColoredPoint **содержит**("has") Point, а также два дополнительных метода Distance и ScaleBy, которые в него пришли после встраивания Point.

Анонимное поле может также быть указателем:

In [2]:
type ColoredPoint struct {
    *Point
    Color color.RGBA
}

var p = ColoredPoint{&Point{3,4}, color.RGBA{0xFF, 0, 0, 0xFF}}
var q = ColoredPoint{&Point{1,1}, color.RGBA{0xFF, 0, 0, 0xFF}}
p.Point = q.Point
fmt.Sprint(*p.Point)

{1 1}


Структура может иметь больше одного анонимного поля:

In [3]:
type ColoredPoint struct {
    *Point
    color.RGBA
}

В этом случае структура ColoredPoint состояла бы из всех методов типов Point и RGBA, а также методов, объявленных именно для неё. Компилятор при вызове p.ScaleBy сначала ищет этот метод в объявленных напрямую для этого типа методах, затем в методах встроенных анонимных полей, затем в методах встроенных полей типов Point и RGBA и т.д. Если два одинаковых метода будут находится на одном уровне, то будет ошибка компиляции.